In [12]:
import os,pickle,sys,numpy,nn,copy,scipy,scipy.io
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import sgd
from keras.layers.recurrent import LSTM
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import SimpleRNN
from keras.initializations import normal, identity
import json
import timeit

start = timeit.default_timer()
# --------------------------------------------
# Parameters
# --------------------------------------------
root_folder_path = '/home/peng/Desktop/ Pynotebooks/Link to Machine_learning_for_reliability_analysis/QSAR/model_weihts/'
seed  = 3453
#print sys.argv
splits = np.arange(0,1)

mb    = 25     # size of the minibatch
hist  = 0.1    # fraction of the history to be remembered

hidden_size = 300

dataset = scipy.io.loadmat('/home/peng/Documents/Project_C/QSAR_nlp/qm7.mat')
for split in splits:
	print "this is the %d split"%split

	# --------------------------------------------
	# Extract training data
	# --------------------------------------------
	P = dataset['P'][range(0,split)+range(split+1,5)].flatten()
	
	x_train = dataset['X'][P]
		
	x_label = dataset['T'][0,P]
	
	print np.shape(x_label)

	# --------------------------------------------
	### Convert it to 1D for irnn
	x_train = x_train.reshape(x_train.shape[0], -1)
	#x_train /= x_train.reshape(x_train.shape[0], -1, 1)
	x_train = x_train.astype('float32')
	x_label = x_label.astype('float32')
	print np.shape(x_train)
	
	# --------------------------------------------
	# Design the DNN model
	# --------------------------------------------


	model = Sequential()
	model.add(Dense(output_dim=hidden_size,
	                    #init=lambda shape, name: normal(shape, scale=0.001, name=name),
	                    #inner_init=lambda shape, name: identity(shape, scale=1.0, name=name),
	                    activation='relu',
	                    input_shape=x_train.shape[1:]))
	model.add(Dense(hidden_size, activation='relu'))
	model.add(Dense(hidden_size, activation='relu'))
	model.add(Dense(1, activation='linear'))
	model.compile(loss="mean_squared_error", optimizer="rmsprop",verbose=False)

	model.fit(x_train, x_label, batch_size=mb, nb_epoch=15),time.sleep(0.2)

   
    
    ###design DNN and fit


	###save the weights

	model.save_weights(root_folder_path + 'nn_keras_%d'%split + ".h5", overwrite=True)	
	with open(root_folder_path + 'nn_keras_%d'%split + ".json", "w") as outfile:		
	    json.dump(model.to_json(), outfile)

stop = timeit.default_timer()

print ("The running takes %r min" %((stop-start)/60))

this is the 0 split
(5732,)
(5732, 529)


ValueError: Invalid argument 'verbose' passed to K.function

In [18]:
import os,pickle,sys,numpy,copy,scipy,scipy.io
import numpy as np
import json
from keras.models import model_from_json
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import sgd
from keras.layers.recurrent import LSTM
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import SimpleRNN
from keras.initializations import normal, identity
import json


root_folder_path = '/home/peng/Desktop/ Pynotebooks/Link to Machine_learning_for_reliability_analysis/QSAR/model_weihts/'
# --------------------------------------------
# Parameters
# --------------------------------------------
#split = int(sys.argv[1]) # test split for cross-validation (between 0 and 5)
splits = np.arange(0,1)
# --------------------------------------------
# Load data and models
# --------------------------------------------
#if not os.path.exists('qm7.mat'): os.system('wget http://www.quantum-machine.org/data/qm7.mat')
dataset = scipy.io.loadmat('/home/peng/Documents/Project_C/QSAR_nlp/qm7.mat')
acc_mae = 0.0

for split in splits:
	with open('/home/peng/Desktop/ Pynotebooks/Link to Machine_learning_for_reliability_analysis/QSAR/model_weihts/nn_keras%d'%split + ".json", 'r') as jfile:
		model = model_from_json(json.load(jfile))
	model.load_weights('/home/peng/Desktop/ Pynotebooks/Link to Machine_learning_for_reliability_analysis/QSAR/model_weihts/nn_keras%d'%split + '.h5')
	model.compile("sgd", "mse")
	
#	print('results after %d iterations'%nn.nbiter)
	
	Ptrain = dataset['P'][range(0,split)+range(split+1,5)].flatten()
	#Ptest  = dataset['P'][split]
	Ptest  = dataset['P'][split-1]
	for P,name in zip([Ptest],['test']):
		# --------------------------------------------
		# Extract test data
		# --------------------------------------------
		x_train = dataset['X'][P]
		x_train = x_train.reshape(x_train.shape[0], -1, 1)
		x_label = dataset['T'][0,P]
	
		# --------------------------------------------
		# Test the neural network
		# --------------------------------------------
		print('\n%s set:'%name)
		predict_label = model.predict(x_train)
        print predict_label
		
		
		
		#print('MAE:  %5.2f kcal/mol'%numpy.abs(np.float(predict_label)-np.float(x_label).mean(axis=0))

print ('finished')              
              


test set:
[[ nan]
 [ nan]
 [ nan]
 ..., 
 [ nan]
 [ nan]
 [ nan]]
finished
